In [ ]:
import yaml
import numpy as np
from sklearn.model_selection import ParameterSampler
from pathlib import Path
import os
from copy import deepcopy

In [ ]:
BASE_CONFIG = {
 'engine_params': None, # <-- tune this
 'include_horizon_day': True,
 'include_horizon_feature': True,
 'include_horizon_month': True,
 'include_horizon_year': True,
 'multi_horizon': True,
 'preprocessing_config': {'data_feature_eng_list': [{'feature_eng_list': [{'kind': 'lag',
      'params': {'lag': 0}},
     {'kind': 'lag', 'params': {'lag': 1}},
     {'kind': 'lag', 'params': {'lag': 2}},
     {'kind': 'lag', 'params': {'lag': 3}},
     {'kind': 'lag', 'params': {'lag': 4}},
     {'kind': 'lag', 'params': {'lag': 5}},
     {'kind': 'lag', 'params': {'lag': 6}},
     {'kind': 'lag', 'params': {'lag': 7}},
     {'kind': 'lag', 'params': {'lag': 11}},
     {'kind': 'lag', 'params': {'lag': 15}},
     {'kind': 'lag', 'params': {'lag': 19}},
     {'kind': 'lag', 'params': {'lag': 25}},
     {'kind': 'lag', 'params': {'lag': 51}},
     {'kind': 'lag', 'params': {'lag': 103}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 4}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 8}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 12}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 26}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 52}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 104}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 4}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 8}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 12}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 26}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 52}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 104}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 4}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 8}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 12}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 26}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 52}},
     {'kind': 'rolling', 'params': {'statistic': 'max', 'window': 104}}],
    'groupby': None,
    'source': 'sales'},
   {'feature_eng_list': [{'kind': 'lag', 'params': {'lag': 0}},
     {'kind': 'lag', 'params': {'lag': 1}},
     {'kind': 'lag', 'params': {'lag': 2}},
     {'kind': 'lag', 'params': {'lag': 3}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 4}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 8}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 12}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 26}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 52}},
     {'kind': 'rolling', 'params': {'statistic': 'mean', 'window': 104}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 4}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 8}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 12}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 26}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 52}},
     {'kind': 'rolling', 'params': {'statistic': 'std', 'window': 104}}],
    'groupby': ['Warehouse'],
    'source': 'sales'},
   {'feature_eng_list': [{'kind': 'lag', 'params': {'lag': 0}},
     {'kind': 'lag', 'params': {'lag': 1}},
     {'kind': 'lag', 'params': {'lag': 2}},
     {'kind': 'lag', 'params': {'lag': 3}},
     {'kind': 'lag', 'params': {'lag': 4}},
     {'kind': 'lag', 'params': {'lag': 5}},
     {'kind': 'lag', 'params': {'lag': 6}},
     {'kind': 'lag', 'params': {'lag': 7}},
     {'kind': 'lag', 'params': {'lag': 11}},
     {'kind': 'lag', 'params': {'lag': 15}},
     {'kind': 'lag', 'params': {'lag': 19}},
     {'kind': 'lag', 'params': {'lag': 25}},
     {'kind': 'lag', 'params': {'lag': 51}},
     {'kind': 'lag', 'params': {'lag': 103}}],
    'groupby': ['Client'],
    'source': 'sales'},
   {'feature_eng_list': [{'kind': 'lag', 'params': {'lag': 0}},
     {'kind': 'lag', 'params': {'lag': 1}},
     {'kind': 'lag', 'params': {'lag': 2}},
     {'kind': 'lag', 'params': {'lag': 3}}],
    'groupby': ['Product'],
    'source': 'sales'}],
  'date_features': ['year', 'month', 'day'],
  'normalize_price': False,
  'normalize_sales': False,
  'static_feature_list': [{'categorical': True, 'name': 'Client'},
   {'categorical': True, 'name': 'Warehouse'},
   {'categorical': True, 'name': 'Product'}]}}

In [ ]:
# Define the hyperparameter space
param_grid = {
    'learning_rate': [0.033932217718953266],
    'n_estimators': [488],
    'max_depth': [11],
    'num_leaves': [110,128,150,180,220,275,350],
    'min_child_samples': [25, 48],
    'subsample': [0.1,0.2,0.33,0.5],
    'colsample_bytree': [0.6],
    'verbose': [0],
}

In [ ]:
# Initialize ParamSampler
n_iter = 56
param_sampler = ParameterSampler(param_grid, n_iter=n_iter, random_state=42)

# Generate the combinations
combinations = list(param_sampler)

In [ ]:
path = Path("../configs/6_tune_mix_hparams")
os.makedirs(path, exist_ok=True)

for i, params in enumerate(combinations):
    config = deepcopy(BASE_CONFIG)
    config["engine_params"] = params
    i = str(i+1)
    i = "0"* (3 - len(i)) + i
    with open(path / f"{i}_config.yaml","w") as f:
        yaml.safe_dump(config,f)